In [2]:
data JSON = JNull
          | JBool Bool
          | JInt Int
          | JFloat Float
          | JString String
          | JArray [JSON]
          | JObject [(String, JSON)]
  deriving Show

  -- | Pretty print a JSON value
prettyJSON :: JSON -> String
prettyJSON JNull       = "null"
prettyJSON (JBool   b) = if b then "true" else "false"
prettyJSON (JInt    i) = show i
prettyJSON (JFloat  f) = show f
prettyJSON (JString s) = '\"' : s ++ "\""
prettyJSON (JArray  a) = '[' : commaSep (map prettyJSON a) ++ "]"
prettyJSON (JObject o) = '{' : commaSep (map prettyKV o) ++ "}"
  where
    prettyKV (k, v) = '\"' : k ++ "\": " ++ prettyJSON v

-- | Transform a list of strings into a comma separated string
commaSep :: [String] -> String
commaSep [] = ""
commaSep s  = foldr1 (\s1 s2 -> s1 ++ ", " ++ s2) s

In [3]:
json1 = JArray [
    JObject [
    ("name", JString "meier"),
    ("besuchte_kurse", JArray [JString "Funktionale Programmierung", JString "Verteilte Systeme", JString "Klöppeln am Nachmittag"]),
    ("note", JNull),
    ("zugelassen", JBool True)
    ],
    JObject [
    ("name", JString "schmidt"),
    ("besuchte_kurse", JArray [JString "Verteilte Systeme", JString "Klöppeln am Nachmittag"]),
    ("note", JFloat 2.7),
    ("zugelassen", JBool False)
    ]
    ]

prettyJSON json1

"[{\"name\": \"meier\", \"besuchte_kurse\": [\"Funktionale Programmierung\", \"Verteilte Systeme\", \"Kl\246ppeln am Nachmittag\"], \"note\": null, \"zugelassen\": true}, {\"name\": \"schmidt\", \"besuchte_kurse\": [\"Verteilte Systeme\", \"Kl\246ppeln am Nachmittag\"], \"note\": 2.7, \"zugelassen\": false}]"

### warum unsere Methode nicht funktioniert
```
test:: (a -> a) -> a -> Int -> JSON -> a
test fun acc i (JInt js) = fun js

test show "" 4
```
das problem ist, dass die JInt immer nen Int als wert hat. das bedeutet, das `a`, welches ich in die funktion reingeben will, wird nicht akzeptiert

In [13]:
foldJSON ::  b -> (Bool -> b) -> (Int -> b) -> (Float -> b) -> (String -> b) -> ([b] -> b) -> ([(String, b)] -> b) -> JSON -> b
foldJSON fn _  _  _  _  _  _     JNull          = fn
foldJSON _  fb _  _  _  _  _    (JBool b)       = fb b
foldJSON _  _  fi _  _  _  _    (JInt  i)       = fi i
foldJSON _  _  _  fl _  _  _    (JFloat f)      = fl f
foldJSON _  _  _  _  fs _  _    (JString s)     = fs s
foldJSON fn fb fi fl fs fa fo   (JArray xs)     = fa (map (foldJSON fn fb fi fl fs fa fo) xs)
foldJSON fn fb fi fl fs fa fo   (JObject xs)    = fo (map (\(k,v) -> (k, foldJSON fn fb fi fl fs fa fo v)) xs)

foldJSON "" show show show show show show json1

Line 9: Use second
Found:
\ (k, v) -> (k, foldJSON fn fb fi fl fs fa fo v)
Why not:
Data.Bifunctor.second (foldJSON fn fb fi fl fs fa fo)

"[\"[(\\\"name\\\",\\\"\\\\\\\"meier\\\\\\\"\\\"),(\\\"besuchte_kurse\\\",\\\"[\\\\\\\"\\\\\\\\\\\\\\\"Funktionale Programmierung\\\\\\\\\\\\\\\"\\\\\\\",\\\\\\\"\\\\\\\\\\\\\\\"Verteilte Systeme\\\\\\\\\\\\\\\"\\\\\\\",\\\\\\\"\\\\\\\\\\\\\\\"Kl\\\\\\\\\\\\\\\\246ppeln am Nachmittag\\\\\\\\\\\\\\\"\\\\\\\"]\\\"),(\\\"note\\\",\\\"\\\"),(\\\"zugelassen\\\",\\\"True\\\")]\",\"[(\\\"name\\\",\\\"\\\\\\\"schmidt\\\\\\\"\\\"),(\\\"besuchte_kurse\\\",\\\"[\\\\\\\"\\\\\\\\\\\\\\\"Verteilte Systeme\\\\\\\\\\\\\\\"\\\\\\\",\\\\\\\"\\\\\\\\\\\\\\\"Kl\\\\\\\\\\\\\\\\246ppeln am Nachmittag\\\\\\\\\\\\\\\"\\\\\\\"]\\\"),(\\\"note\\\",\\\"2.7\\\"),(\\\"zugelassen\\\",\\\"False\\\")]\"]"